In [ ]:
import numpy as np
import pyvista as pv
import shapely as sh
import matplotlib.pyplot as plt

from airfoil.util.shapely_helpers import plot_shapely_directional
from airfoil.util.linestring_helpers import resample_shapes
from airfoil.util.pyvista_helpers import make_mesh_from_side_surfaces
from airfoil.cnc.cnc_machine_mesh import draw_machine
from airfoil.cnc import (
    states_to_3d_points,
    state_to_3d_points
)

# Build a toy mesh to stress test Path finding Algorithms

In [ ]:
# create components
s0 = sh.Point(0,0).buffer(60)
s1 = sh.Point(0,0).buffer(60+30)
s2 = sh.box(-100,-100,100,0)
s3 = sh.box(-15,50, 15,250)
clip_circle = sh.Point(0,100).buffer(90)

# combine scale and rotate them a bit to make an interesting spork-like shape to play with
shapea_p = sh.affinity.scale(
    sh.difference(sh.union(s3,sh.difference(s1,s2)),s0),
    0.5,0.5, origin=(0,0)
)
shapeb_p = sh.affinity.rotate(sh.intersection(shapea_p,clip_circle),10)

# resample the shape outlines
# we need them to be
shapea,shapeb = resample_shapes(
    [
        np.array(shapea_p.exterior.coords),
        np.array(shapeb_p.exterior.coords),
    ],
    target_length=5
)

shapea_p, shapeb_p = sh.Polygon(shapea),sh.Polygon(shapeb)
shapec_p, shaped_p = [
    sh.affinity.translate(sh.affinity.rotate(sh.Polygon(shapeb),170), 30,-120),
    sh.affinity.translate(sh.affinity.rotate(sh.Polygon(shapea),190), 50,-100),
]


plot_shapely_directional([
        shapea_p, 
        shapeb_p, 
        shapec_p, 
        shaped_p
    ],
    legend=["a","b","c","d"]
)

# Plot a Sample of 4D Space

In [ ]:
# Show a tiny sample of 4D space
meesh = make_mesh_from_side_surfaces(shapea, shapeb)
meesh2 = make_mesh_from_side_surfaces(
    np.array(shapec_p.exterior.coords),
    np.array(shaped_p.exterior.coords),
)

(_,_,yl,yu,zl,zu) = (meesh+meesh2).bounds

x = np.random.uniform(yl-5, yu+5, 1000)
y = np.random.uniform(zl-5, zu+5, 1000)
z = np.random.uniform(yl-5, yu+5, 1000)
a = np.random.uniform(zl-5, zu+5, 1000)

aa,bb = states_to_3d_points(x,y,z,a)

combo = (meesh+meesh2)
rez = []
for aai,bbi in zip(aa,bb):
    pnt,cell = combo.ray_trace(aai,bbi)
    rez.append(len(pnt)==0)
aa = aa[rez]
bb = bb[rez]

pt = pv.Plotter()
for aai,bbi in zip(aa,bb):
    pt.add_mesh(pv.Line(aai,bbi), color="pink", opacity=0.5)


target_state = np.array([yu+2,0,yu+2,0])

pt.add_mesh(pv.Line(*state_to_3d_points(*target_state)), color="blue")

pt.add_mesh(meesh)
pt.add_mesh(meesh2)
draw_machine(-10,0,-10,0,spacing=220, pt=pt, opacity=0.1)
pt.show()

# Spitfire Wing

More realistic example of a spitfire wing sliced into segments and stacked

In [ ]:
from airfoil import Decomposer
from airfoil.examples.spitfire import SpitfireWing
wing = SpitfireWing(750)

section_positions = [0, 200, 400, 500, 600, 650, 700, 720]
segments = wing.create_segments(section_positions=section_positions)

import pyvista as pv
pt = pv.Plotter()
offset = 0
for segment in segments:
    offset+=segment.length/2
    pt.add_mesh(segment.to_mesh(Decomposer(segment_target_length=5)).translate((offset,0,0)), show_edges=True)
    offset+=segment.length/2
pt.show()

In [ ]:
from airfoil.cnc.cnc_machine_mesh import draw_machine
import pyvista as pv
pt = pv.Plotter()#window_size=(1000, 1000))
offset_z = 0
flip_flop = False
existing_meshes = []
lalign = 0
for segment in segments:
    flip_flop = not flip_flop
    segmesh = segment.to_mesh(Decomposer(segment_target_length=5))
    if flip_flop:
        segmesh = segmesh.rotate_y(180)
    zcen = (segmesh.bounds.z_max+segmesh.bounds.z_min)/2
    lalign   = min(lalign, segmesh.bounds.x_min)
    segmesh = segmesh.translate((-(lalign-segmesh.bounds.x_min),0,-zcen))
    for last_mesh in existing_meshes:
        while True:
            _,numcols = segmesh.collision(last_mesh, contact_mode=1)
            if numcols == 0:
                segmesh = segmesh.translate((0,0,4))
                break
            else:
                segmesh = segmesh.translate((0,0,2))
    existing_meshes.append(segmesh)
    pt.add_mesh(segmesh, show_edges=True, edge_color="#333333")
draw_machine(0,5,0,30,360,pt=pt)
#pt.enable_shadows()
pt.show()

In [ ]:
from sklearn.svm import SVC
import numpy as np
m0 = existing_meshes[2]
m1 = existing_meshes[3]
X = np.concat([m0.points,m1.points],axis=0)
y = np.concat([np.zeros(len(m0.points)), np.ones(len(m1.points))])
SVC(kernel="linear").fit(X,y)


# Find plane to separate shapes using a Support Vector Machine to find a Separating Plane

In this example we do things in 3D... it is interesting to wonder if a similar thing 
can be done for the stress test shapes in 4D?

In [ ]:
from sklearn.svm import SVC
import numpy as np
import pyvista as pv

# Your existing code
m0 = existing_meshes[2]
m1 = existing_meshes[3]
X = np.concatenate([m0.points, m1.points], axis=0)  # Fixed: concat -> concatenate
y = np.concatenate([np.zeros(len(m0.points)), np.ones(len(m1.points))])

# Train SVM with linear kernel to get hyperplane
svm = SVC(kernel='linear')
svm.fit(X, y)


# Get hyperplane parameters
normal = svm.coef_[0]  # Direction (normal vector)
bias = svm.intercept_[0]

# Calculate center point on the hyperplane
# Project origin onto hyperplane and adjust for bias
center = -bias * normal / np.dot(normal, normal)

print(f"Center: {center}")
print(f"Direction (normal): {normal}")

# For PyVista: create a simple plane
import pyvista as pv
plane = pv.Plane(center=center, direction=normal, i_size=500, j_size=500)  #

pt = pv.Plotter()
pt.add_mesh(m0)
pt.add_mesh(m1)
pt.add_mesh(plane, color="red", opacity=0.5)
pt.show()


# A* Path Finding Algorithm

Here we vibe coded with claude to attempt a quick A* implementation in 4D.

Even with 5mm steps this doesn't work. Every grid position has 27 neighbors, so the number of cells to track gets stupid very quickly.

This may still be feasible for low resolution grids and some native rust code... but i have a feeling there may be better approaches than A* anyway?

Claud's implementation here is riddled with unnecessary branching and bad python slowness... perhaps it could be improved... dunno if it is worth it.

In [ ]:
# Vibe Coded by Claude. As expected A* does not work as expected. 
# This doesnt look like an efficient implementation tho... so its probably maybe possible.
# 4D is just insanely difficult to path-find in
import numpy as np
from typing import Iterable, Tuple
import pyvista as pv
import time

# Assuming your AStar library is imported
from astar import find_path

class StateSpacePathfinder:
    def __init__(self, mesh_combo, bounds, step_size=5.0, coordination_weight=2.0):
        self.mesh_combo = mesh_combo
        self.bounds = bounds  # (xl, xu, yl, yu, zl, zu)
        self.step_size = step_size
        self.coordination_weight = coordination_weight  # Penalty for XY-ZA deviation
        
        # Progress tracking
        self.iteration_count = 0
        self.start_time = None
        self.visited_states = set()
        
    def is_valid_state(self, state: Tuple[float, float, float, float]) -> bool:
        """Check if state is within bounds and collision-free"""
        x, y, z, a = state
        xl, xu, yl, yu, zl, zu = self.bounds
        
        # Bounds check
        if not (xl <= x <= xu and yl <= y <= yu and zl <= z <= zu and zl <= a <= zu):
            return False
            
        # Collision check using your existing function
        try:
            aa, bb = states_to_3d_points([x], [y], [z], [a])
            pnt, cell = self.mesh_combo.ray_trace(aa[0], bb[0])
            return len(pnt) == 0  # No collision if no intersection points
        except:
            return False
    
    def get_neighbors(self, state: Tuple[float, float, float, float]) -> Iterable[Tuple[float, float, float, float]]:
        """Generate valid neighboring states with progress tracking"""
        self.iteration_count += 1
        self.visited_states.add(state)
        
        # Progress reporting every 1000 iterations
        if self.iteration_count % 1000 == 0:
            self._report_progress()
        
        x, y, z, a = state
        
        # 4D movement directions (80 neighbors in 4D)
        directions = []
        for dx in [-self.step_size, 0, self.step_size]:
            for dy in [-self.step_size, 0, self.step_size]:
                for dz in [-self.step_size, 0, self.step_size]:
                    for da in [-self.step_size, 0, self.step_size]:
                        if dx == dy == dz == da == 0:
                            continue
                        directions.append((dx, dy, dz, da))
        
        neighbors = []
        for dx, dy, dz, da in directions:
            new_state = (x + dx, y + dy, z + dz, a + da)
            if self.is_valid_state(new_state):
                neighbors.append(new_state)
        
        return neighbors
    
    def _report_progress(self):
        """Report pathfinding progress"""
        if self.start_time is None:
            self.start_time = time.time()
        
        elapsed = time.time() - self.start_time
        states_per_sec = self.iteration_count / elapsed if elapsed > 0 else 0
        
        print(f"Progress Report - Iteration {self.iteration_count:,}")
        print(f"  Elapsed time: {elapsed:.1f}s")
        print(f"  States explored: {len(self.visited_states):,}")
        print(f"  Exploration rate: {states_per_sec:.1f} states/sec")
        print(f"  Open/closed ratio: {self.iteration_count / len(self.visited_states):.2f}")
        print()
    
    def heuristic(self, current: Tuple[float, float, float, float], 
                  goal: Tuple[float, float, float, float]) -> float:
        """
        Modified heuristic: XY distance + ZA distance + coordination penalty
        """
        cx, cy, cz, ca = current
        gx, gy, gz, ga = goal
        
        # Separate XY and ZA distances
        xy_distance = np.sqrt((cx - gx)**2 + (cy - gy)**2)
        za_distance = np.sqrt((cz - gz)**2 + (ca - ga)**2)
        
        # Base heuristic is sum of XY and ZA distances
        base_cost = xy_distance + za_distance
        
        # Coordination penalty: encourage XY and ZA to move in lockstep
        # Calculate how far apart the normalized distances are
        if xy_distance > 0 and za_distance > 0:
            max_dist = max(xy_distance, za_distance)
            min_dist = min(xy_distance, za_distance)
            coordination_penalty = (max_dist - min_dist) * self.coordination_weight
        else:
            coordination_penalty = 0
        
        return base_cost + coordination_penalty
    
    def distance(self, state1: Tuple[float, float, float, float], 
                 state2: Tuple[float, float, float, float]) -> float:
        """
        Modified distance: XY movement cost + ZA movement cost + coordination penalty
        """
        x1, y1, z1, a1 = state1
        x2, y2, z2, a2 = state2
        
        # Separate XY and ZA movements
        xy_move = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        za_move = np.sqrt((z2 - z1)**2 + (a2 - a1)**2)
        
        # Base cost is sum of XY and ZA movements
        base_cost = xy_move + za_move
        
        # Coordination penalty for unbalanced movements
        if xy_move > 0 and za_move > 0:
            max_move = max(xy_move, za_move)
            min_move = min(xy_move, za_move)
            coordination_penalty = (max_move - min_move) * self.coordination_weight
        elif xy_move > 0 and za_move == 0:
            # Penalty for moving only in XY
            coordination_penalty = xy_move * self.coordination_weight * 0.5
        elif za_move > 0 and xy_move == 0:
            # Penalty for moving only in ZA
            coordination_penalty = za_move * self.coordination_weight * 0.5
        else:
            coordination_penalty = 0
        
        return base_cost + coordination_penalty

# Main pathfinding function
def find_4d_path(start_state, target_state, mesh_combo, bounds, step_size=1.0, coordination_weight=2.0):
    """
    Find A* path in 4D state space with improved cost structure
    
    Args:
        start_state: Starting 4D state (x, y, z, a)
        target_state: Target 4D state (x, y, z, a)
        mesh_combo: Combined mesh for collision detection
        bounds: Search space bounds (xl, xu, yl, yu, zl, zu)
        step_size: Step size for movement
        coordination_weight: Weight for penalizing uncoordinated XY-ZA movement
    """
    
    pathfinder = StateSpacePathfinder(mesh_combo, bounds, step_size, coordination_weight)
    
    # Convert numpy arrays to tuples for hashing
    start = tuple(start_state)
    goal = tuple(target_state)
    
    print(f"Starting 4D pathfinding...")
    print(f"  Start: {start}")
    print(f"  Goal: {goal}")
    print(f"  Step size: {step_size}")
    print(f"  Coordination weight: {coordination_weight}")
    print(f"  Search bounds: {bounds}")
    print()
    
    pathfinder.start_time = time.time()
    
    # Use the provided find_path function
    path = find_path(
        start=start,
        goal=goal,
        neighbors_fnct=pathfinder.get_neighbors,
        heuristic_cost_estimate_fnct=pathfinder.heuristic,
        distance_between_fnct=pathfinder.distance,
        is_goal_reached_fnct=lambda current, target: np.linalg.norm(np.array(current) - np.array(target)) < step_size
    )
    
    # Final progress report
    total_time = time.time() - pathfinder.start_time
    print(f"\nPathfinding completed in {total_time:.1f}s")
    print(f"Total iterations: {pathfinder.iteration_count:,}")
    print(f"States explored: {len(pathfinder.visited_states):,}")
    
    return list(path) if path else None

# Enhanced usage with your existing code:
if __name__ == "__main__":
    # Get bounds from your mesh
    combo = (meesh + meesh2)
    xl, xu, yl, yu, zl, zu = combo.bounds
    
    # Extend bounds slightly for search space
    bounds = (xl-5, xu+5, yl-5, yu+5, zl-5, zu+5)
    
    # Find path with coordination preference
    start_state = np.array([0, 0, 0, 0])
    path = find_4d_path(
        start_state, 
        target_state, 
        combo, 
        bounds, 
        step_size=2.0,
        coordination_weight=3.0  # Higher values encourage more coordinated movement
    )
    
    if path:
        print(f"\nPath found with {len(path)} waypoints")
        
        # Analyze path coordination
        xy_distances = []
        za_distances = []
        for i in range(len(path) - 1):
            x1, y1, z1, a1 = path[i]
            x2, y2, z2, a2 = path[i + 1]
            xy_dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            za_dist = np.sqrt((z2 - z1)**2 + (a2 - a1)**2)
            xy_distances.append(xy_dist)
            za_distances.append(za_dist)
        
        avg_xy = np.mean(xy_distances)
        avg_za = np.mean(za_distances)
        coordination_ratio = min(avg_xy, avg_za) / max(avg_xy, avg_za) if max(avg_xy, avg_za) > 0 else 1.0
        
        print(f"Path analysis:")
        print(f"  Average XY movement: {avg_xy:.2f}")
        print(f"  Average ZA movement: {avg_za:.2f}")
        print(f"  Coordination ratio: {coordination_ratio:.2f} (1.0 = perfect coordination)")
        
        # Visualize the path
        pt = pv.Plotter()
        
        # Add existing meshes
        pt.add_mesh(meesh)
        pt.add_mesh(meesh2)
        
        # Add path visualization
        for i, state in enumerate(path):
            aa_path, bb_path = states_to_3d_points([state[0]], [state[1]], [state[2]], [state[3]])
            color = "red" if i == 0 else "green" if i == len(path)-1 else "yellow"
            pt.add_mesh(pv.Line(aa_path[0], bb_path[0]), color=color, line_width=3)
        
        # Add target
        pt.add_mesh(pv.Line(*state_to_3d_points(*target_state)), color="blue")
        
        # Add machine visualization
        draw_machine(-10, 0, -10, 0, spacing=220, pt=pt, opacity=0.1)
        
        pt.show()
    else:
        print("No path found")